In [2]:
import shutil

f = shutil.copy
print(f)
f = shutil.move
print(f)

<function copy at 0x7f8a6e482f80>
<function move at 0x7f8a6e4835b0>


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from source.utils.conformer_generation import generate_conformers
from source.utils.mol2pyg import mol2pyg
from source.data_transforms._frad_transforms import *
import numpy as np
from rdkit import Chem
from copy import deepcopy
from source.utils.mol_utils import set_coords, get_rdkit_conformer
from source.utils.mol_utils import visualize_3d_mols,get_energy
import matplotlib.pyplot as plt

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (/home/nobilm@usi.ch/miniconda3/envs/rdkit_pyg_geqm9_clone/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [ ]:
# smiles = ['[H]c1c([H])c([H])c(N(C(=O)C([H])([H])C([H])([H])[H])C2([H])C([H])([H])C([H])([H])N(C([H])([H])C([H])([H])c3c([H])c([H])c([H])c([H])c3[H])C([H])([H])C2([H])[H])c([H])c1[H]']
smiles = ['C1=CC=CC=C1']
m = Chem.MolFromSmiles(smiles[0])
m = Chem.AddHs(m)
c = get_rdkit_conformer(m)
pygm = mol2pyg(m)
from source.data_transforms._frad_transforms import frad

In [ ]:
frad(pygm)

In [ ]:
# smiles = ['CCCCCC']
smiles = ['[H]c1c([H])c([H])c(N(C(=O)C([H])([H])C([H])([H])[H])C2([H])C([H])([H])C([H])([H])N(C([H])([H])C([H])([H])c3c([H])c([H])c([H])c([H])c3[H])C([H])([H])C2([H])[H])c([H])c1[H]']
data_list = []
for s in smiles: conformers = generate_conformers(s, 11000)
conformers_old = deepcopy(conformers)
max_energy = 2*max(get_energy(m) for m in conformers)
len(conformers), max_energy
# visualize_3d_mols(conformers[:10])

In [ ]:
m = conformers[0]
data = mol2pyg(m, max_energy)
old_data = deepcopy(data)

data_noised = frad(data, dihedral_scale=100) # acts inplace on data
noised_mol = set_coords(m, data_noised.pos)

# old_data.pos == data_noised.pos

In [ ]:
visualize_3d_mols([m, noised_mol])

In [ ]:
original_mols = []
noised_mols = []
max_energy = 2*max(get_energy(m) for m in conformers)
print(max_energy)
for m in conformers:
  data = mol2pyg(m, max_energy)

  data_old = deepcopy(data)
  mol_old = deepcopy(m)
  original_mols.append(mol_old)

  data_noised = frad(data) # acts inplace
  noised_mol = set_coords(m, data_noised.pos)

  noised_mols.append(noised_mol)

In [ ]:
og_conformers_energy = [get_energy(m) for m in original_mols]
og_noised_energy = [get_energy(m) for m in noised_mols]

In [ ]:
og_conformers_energy == og_noised_energy

In [ ]:
len(og_conformers_energy)

In [ ]:
min(og_conformers_energy), max(og_conformers_energy), np.mean(og_conformers_energy), np.std(og_conformers_energy), np.median(og_conformers_energy), np.sum(np.array(og_conformers_energy)<150)

In [ ]:
k = 3
np.mean(og_conformers_energy) + k*(np.abs(min(og_conformers_energy) - max(og_conformers_energy)))

In [ ]:
2*np.max(og_conformers_energy) # ! criterio x filtrare mol wrt energy, 0, 90

In [ ]:
len(og_noised_energy)

In [ ]:
min(og_noised_energy), max(og_noised_energy), np.mean(og_noised_energy), np.std(og_noised_energy), np.median(og_noised_energy), np.sum(np.array(og_noised_energy)<100)
# np.random.normal(0, 90) : (69.89637533061749, 17655.266923111583, 836.6925485529661, 2499.3302193585287)
# unif(0,360): (62.6450572083416, 1338561.7658836602, 33822.30710208779, 172727.59413754893)

In [ ]:
plt.boxplot([og_conformers_energy, og_noised_energy]),
plt.title('Box Plot of Two Different Lists')
plt.xlabel('Sampling from uniform(0, 360)')
plt.ylabel('Energy')
plt.xticks([1, 2], ['og', 'noised'])
plt.grid(True)
plt.show()

In [ ]:
# m = confs[0]
# data = mol2pyg(m)
# data_old = deepcopy(data)
# d1 = frad(data, dihedral_noise_tau=55.0,coords_noise_tau=0.0) # ! Cauchy
# d1.pos == data_old.pos
# m_old = set_coords(m, data_old.pos)
# m_new = set_coords(m, d1.pos)

In [ ]:
# visualize_3d_mols([m_old])

In [ ]:
# visualize_3d_mols([m_new])

# recall that the noise ur sampling gets added to the og value, furthermore think how this impacts the loss

In [ ]:
scale = 40
s = np.random.normal(0, scale, size=1000000)
s = np.clip(s, a_min=-180., a_max=180)
print(min(s), max(s), np.std(s))
plt.hist(s, bins=1000)
plt.show()